In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from gpt_model import GPT
from vocab import tokens
import torch.nn as nn
import random

In [31]:
all_tokens = [
    ["eu",     "vou",   "abrir",   "a",       "porta",     "agora"],
    ["ela",    "querer","comprar", "um",      "livro",     "na",       "livraria"],
    ["eles",   "vão",   "correr",  "na",      "praia",     "amanhã"],
    ["nós",    "estamos","felizes","porque",  "ganhar",    "o",        "jogo"],
    ["você",   "poder", "trazer",  "água",    "agora"],
    ["ele",    "vai",   "estudar", "história","na",        "noite"],
    ["elas",   "dever", "proteger","a",       "floresta",  "sempre"],
    ["eu",     "precisar","dormir","bem",     "hoje"],
    ["vocês",  "podem", "usar",    "este",    "caminho"],
    ["nós",    "vamos", "viajar",  "para",    "a",         "cidade",   "amanhã"],
    ["ele",    "querer","beber",   "café",    "agora"],
    ["ela",    "vai",   "abrir",   "um",      "projeto",   "novo"],
    ["eles",   "precisar","resolver","o",     "caso",      "hoje"],
    ["eu",     "vou",   "ler",     "a",       "mensagem",  "agora"],
    ["vocês",  "dever", "chegar",  "antes",   "das",       "dez"],
    ["nós",    "vamos", "buscar",  "ajudar"],
    ["ele",    "vai",   "jogar",   "bola",    "amanhã"],
    ["ela",    "gostar","de",      "música",  "sempre"],
    ["eu",     "vou",   "andar",   "no",      "caminho",   "verde"],
    ["eles",   "vão",   "trazer",  "amor",    "para",      "casa"],
    ["você",   "precisar","estudar","bem",    "agora"],
    ["nós",    "estamos","aprender","agora"],
    ["eles",   "querer","subir",   "a",       "montanha",  "amanhã"],
    ["ela",    "vai",   "assistir","filme",   "hoje"],
    ["eu",     "vou",   "olhar",   "o",       "céu",       "claro"],
    ["vocês",  "podem", "brincar", "com",     "o",         "gato"],
    ["ele",    "precisar","ligar", "para",    "o",         "aluno"],
    ["nós",    "vamos", "fazer",   "evento",  "amanhã"],
    ["ela",    "querer","ganhar",  "dinheiro","rápido"],
    ["eles",   "vão",   "mudar",   "de",      "casa"],
    ["eu",     "precisar","tomar", "água",    "agora"],
    ["ela",    "vai",   "voltar",  "para",    "casa",      "logo"],
    ["vocês",  "dever", "seguir",  "a",       "lei"],
    ["nós",    "vamos", "sugerir", "ideia",   "melhor"],
    ["eles",   "vão",   "amar",    "a",       "família"],
    ["eu",     "querer","receber", "uma",     "mensagem",  "logo"],
    ["ela",    "precisar","resolver","caso",  "difícil"],
    ["vocês",  "podem", "reduzir", "tempo",   "agora"],
    ["nós",    "vamos", "produzir","um",      "novo",      "sistema"],
    ["eles",   "querer","formar",  "empresa", "forte"],
    ["eu",     "vou",   "escrever","texto",   "rápido"],
    ["ela",    "vai",   "buscar",  "ideia",   "melhor"],
    ["vocês",  "podem", "assistir","jogo",    "hoje"],
    ["nós",    "vamos", "encontrar","o",      "grupo",     "amanhã"],
    ["eles",   "vão",   "viajar",  "este",    "mês"],
    ["eu",     "vou",   "abrir",   "o",       "jogo",      "agora"],
    ["ela",    "querer","viver",   "vida",    "livre"],
    ["vocês",  "dever", "seguir",  "com",     "o",         "projeto"],
    ["nós",    "vamos", "rir",     "da",      "história"],
    ["eles",   "vão",   "sair",    "da",      "cidade"]
]


In [32]:
# Inicializa as listas de inputs e targets
inputs, targets = [], []

# Para cada frase, acumula pares (contexto → próximo token)
for tokens in all_tokens:
    for i in range(1, len(tokens)):
        # contexto: tokens até a posição i (excluindo i)
        inputs.append(tokens[:i])
        print(tokens[:i])
        # target: token na posição i
        targets.append(tokens[i])
        print(tokens[i])

['eu']
vou
['eu', 'vou']
abrir
['eu', 'vou', 'abrir']
a
['eu', 'vou', 'abrir', 'a']
porta
['eu', 'vou', 'abrir', 'a', 'porta']
agora
['ela']
querer
['ela', 'querer']
comprar
['ela', 'querer', 'comprar']
um
['ela', 'querer', 'comprar', 'um']
livro
['ela', 'querer', 'comprar', 'um', 'livro']
na
['ela', 'querer', 'comprar', 'um', 'livro', 'na']
livraria
['eles']
vão
['eles', 'vão']
correr
['eles', 'vão', 'correr']
na
['eles', 'vão', 'correr', 'na']
praia
['eles', 'vão', 'correr', 'na', 'praia']
amanhã
['nós']
estamos
['nós', 'estamos']
felizes
['nós', 'estamos', 'felizes']
porque
['nós', 'estamos', 'felizes', 'porque']
ganhar
['nós', 'estamos', 'felizes', 'porque', 'ganhar']
o
['nós', 'estamos', 'felizes', 'porque', 'ganhar', 'o']
jogo
['você']
poder
['você', 'poder']
trazer
['você', 'poder', 'trazer']
água
['você', 'poder', 'trazer', 'água']
agora
['ele']
vai
['ele', 'vai']
estudar
['ele', 'vai', 'estudar']
história
['ele', 'vai', 'estudar', 'história']
na
['ele', 'vai', 'estudar', 'hist

In [33]:
train_data = list(zip(inputs, targets))
random.shuffle(train_data)

In [34]:
# ── 1. modelo + otimizador ───────────────────────────────────────────────
#device      = "cuda" if torch.cuda.is_available() else "cpu"
context_length = 20
n_layers       = 3
n_heads        = 2
n_embd         = 64
max_tokens     = 12
model       = GPT(context_length, n_layers, n_heads, n_embd, max_tokens)
optimizer   = AdamW(model.parameters(), lr=1e-3)
criterion   = nn.CrossEntropyLoss()

In [35]:
# ── 2. loop de treino ────────────────────────────────────────────────────
num_epochs   = 15
loss_history = []

for epoch in range(num_epochs):
    epoch_loss = 0.0

    for input_tokens, target_token in train_data:          # ↳ seu dataloader/iterável
        # (opcional) envie tudo para GPU, se estiver usando
        input_tokens = list(input_tokens)                  # garante lista mutável
        model.init_tokens(input_tokens)                    # define contexto atual

        # transforma tokens em índices para o forward
        # ── forward ──────────────────────────────────────
        idx = model._update_indices_tensor(input_tokens)
        logits = model.forward(idx)                    # (n_embd,)
        #logits      = model.predict_logits(last_hidden)    # (vocab_size,)

        # ── loss ────────────────────────────────────────
        target_idx  = torch.tensor([model.token_to_idx[target_token]],
                                   dtype=torch.long,
                                   device=logits.device)   # shape (1,)
        loss        = criterion(logits.unsqueeze(0), target_idx)

        # ── backward + update ───────────────────────────
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # média da loss na época
    avg_loss = epoch_loss / len(train_data)
    loss_history.append(avg_loss)
    print(f"Época {epoch+1}/{num_epochs} — Loss: {avg_loss:.4f}")


Época 1/15 — Loss: 5.6971
Época 2/15 — Loss: 3.7109
Época 3/15 — Loss: 2.9089
Época 4/15 — Loss: 2.2730
Época 5/15 — Loss: 1.7948
Época 6/15 — Loss: 1.4556
Época 7/15 — Loss: 1.2614
Época 8/15 — Loss: 1.0583
Época 9/15 — Loss: 0.9418
Época 10/15 — Loss: 0.8648
Época 11/15 — Loss: 0.8208
Época 12/15 — Loss: 0.7751
Época 13/15 — Loss: 0.7445
Época 14/15 — Loss: 0.7184
Época 15/15 — Loss: 0.7018


In [36]:
input = "nós vamos"

In [37]:
model.init_tokens(input.split())

In [38]:
model.predict_next_token()

'mudar'

In [46]:
model.predict_all_sentence()

Passo 1: nós vamos
Passo 2: nós vamos rir
Passo 3: nós vamos rir da
Passo 4: nós vamos rir da história
Passo 5: nós vamos rir da história amanhã
Passo 6: nós vamos rir da história amanhã produzir
Passo 7: nós vamos rir da história amanhã produzir sugerir
Passo 8: nós vamos rir da história amanhã produzir sugerir ideia
Passo 9: nós vamos rir da história amanhã produzir sugerir ideia sugerir
Passo 10: nós vamos rir da história amanhã produzir sugerir ideia sugerir gostar
Passo 11: nós vamos rir da história amanhã produzir sugerir ideia sugerir gostar de
Passo 12: nós vamos rir da história amanhã produzir sugerir ideia sugerir gostar de casa


['nós',
 'vamos',
 'rir',
 'da',
 'história',
 'amanhã',
 'produzir',
 'sugerir',
 'ideia',
 'sugerir',
 'gostar',
 'de',
 'casa',
 'logo']

In [ ]:
loss = nn.CrossEntropyLoss()
loss(torch.tensor([[0.1, 2.9]]), torch.tensor([1]))

In [ ]:
num_epochs = 2

In [ ]:
# 1) Converte tokens → índices
inputs_idx  = [[vocab[t] for t in seq] for seq in inputs]
targets_idx = [vocab[t]          for t in targets]


X = [pad_truncate(seq, context_length, pad_id) for seq in inputs_idx]
Y = targets_idx  # cada Y[i] já é um escalar

# 3) Tensores e DataLoader
X = torch.tensor(X, dtype=torch.long)   # (N, context_length)
Y = torch.tensor(Y, dtype=torch.long)   # (N,)
ds = TensorDataset(X, Y)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True)

# 4) Otimizador
optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

# 5) Scheduler com warmup + linear decay
def lr_lambda(current_step):
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return max(
        0.0,
        float(total_steps - current_step) / float(max(1, total_steps - warmup_steps))
    )

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


In [ ]:

# 6) Loop de treinamento
model.train()
for epoch in range(num_epochs):
    for step, (batch_x, batch_y) in enumerate(loader):
        # forward
        model.tokens_idx([ idx_to_token[id.item()] for id in batch_x[0] ])  # adapte se necessário
        logits = model.forward()        # retorna (n_embd,), se seu forward for assim ajuste
        # aqui, se seu forward retorna apenas último token, use batchization:
        # logits = model(batch_x)  # se tiver implementado batch no forward
        # assume logits: (batch_size, vocab_size) ou (vocab_size,) + reshape
        # loss
        loss = F.cross_entropy(logits.view(-1, model.vocab_size),
                               batch_y.view(-1),
                               ignore_index=pad_id)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

        #if step % 50 == 0:
        print(f"Epoch {epoch+1} | Step {step}/{len(loader)} | Loss {loss.item():.4f}")

    # opcional: validação aqui

# ao final, salve checkpoint
torch.save({
    "model": model.state_dict(),
    "optimizer": optimizer.state_dict(),
    "epoch": epoch,
    "scheduler": scheduler.state_dict(),
}, "checkpoint.pth")


# Treinamento